In [9]:
import pickle

In [10]:
!nvidia-smi

Wed May 11 01:23:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 8000     On   | 00000000:06:00.0 Off |                    0 |
| N/A   32C    P8    14W / 250W |      0MiB / 45556MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
file = open("/scratch/sm9669/nabird pickle/nabird/train_1000.pkl",'rb')
object_file = pickle.load(file)

In [12]:
print(object_file['0619/0ba9dea32952498baff7a85078804138.jpg'].shape)

(651, 1024, 3)


In [13]:
from tqdm import tqdm
path_to_image = {}
for i in tqdm(range(0, 23)):
    path = '/scratch/sm9669/nabird pickle/nabird/train_'+str(i*1000)+'.pkl'
    file = open(path,'rb')
    sub_data = pickle.load(file)
    path_to_image.update(sub_data)
   

100%|██████████| 23/23 [00:41<00:00,  1.82s/it]


In [14]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 60.5 MB 32.4 MB/s eta 0:00:01    |▍                               | 788 kB 32.4 MB/s eta 0:00:02        | 16.5 MB 32.4 MB/s eta 0:00:02[K     |██████████████████▉             | 35.5 MB 32.4 MB/s eta 0:00:01�█████████████████████████▋| 59.7 MB 32.4 MB/s eta 0:00:01
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


In [14]:
import torch.utils.data as data
from PIL import Image
import os
import json
from torchvision import transforms
import random
import numpy as np
import json
import torch
import pickle
from tqdm import tqdm
import cv2

def default_loader(path):
    image= path_to_image[path]
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
#     return Image.open(path).convert('RGB')
    return im_pil


class NABirds(data.Dataset):
    def __init__(self, root, meta_data, split_name, im_size_resize, im_size_crop, is_train=True, meta_type='ebird_meta'):

        # load annotations
        print('Loading annotations from: ' + os.path.basename(meta_data))
        with open(meta_data) as fp:
            da = json.load(fp)

        # only choose images that are available - ie some missing from download
        # also select the split of interest
        da = [dd for dd in da[split_name] if dd['valid_image']]

        # set up the filenames and annotations
        self.imgs = [dd['im_path'] for dd in da]
        self.classes = [dd['class_id'] for dd in da]
        self.users = [dd[meta_type]['user_id'] for dd in da]
        self.dates = [dd[meta_type]['date'] for dd in da]
        self.lon = [dd[meta_type]['lon'] for dd in da]
        self.lat = [dd[meta_type]['lat'] for dd in da]

        # print out some stats
#         print split_name
#         print '\t' + str(len(self.imgs)) + ' images'
#         print '\t' + str(len(set(self.classes))) + ' classes'

        self.root = root
        self.is_train = is_train
        self.loader = default_loader
        self.im_size_resize = im_size_resize
        self.im_size_crop = im_size_crop

        # augmentation params
        self.mu_data = [0.485, 0.456, 0.406]
        self.std_data = [0.229, 0.224, 0.225]
        self.brightness = 0.4
        self.contrast = 0.4
        self.saturation = 0.4
        self.hue = 0.25

        # augmentations
        self.resize = transforms.Resize(self.im_size_resize)
        self.resize_rand = transforms.RandomResizedCrop(self.im_size_crop)
        self.center_crop = transforms.CenterCrop(self.im_size_crop)
        self.flip_aug = transforms.RandomHorizontalFlip()
        self.color_aug = transforms.ColorJitter(self.brightness, self.contrast, self.saturation, self.hue)
        self.tensor_aug = transforms.ToTensor()
        self.norm_aug = transforms.Normalize(mean=self.mu_data, std=self.std_data)
        

    def __getitem__(self, index):
        path = self.imgs[index]
        im_id = self.imgs[index]
        img = self.loader(path)
        class_id = self.classes[index]

        if self.is_train:
            img = self.resize_rand(img)
            img = self.flip_aug(img)
            img = self.color_aug(img)
        else:
            img = self.resize(img)
            img = self.center_crop(img)

        img = self.tensor_aug(img)
        img = self.norm_aug(img)
        
        return img, im_id, class_id

    def __len__(self):
        return len(self.imgs)

In [17]:
meta_data = '/scratch/sm9669/nabirds_with_loc_2019.json'
data_root = '/nabirds/images/'

In [18]:
train_dataset = NABirds(data_root, meta_data,
                     split_name='train', im_size_crop=299,
                     im_size_resize=342, is_train=True)
    
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64,
               shuffle=True, num_workers=0, pin_memory=True)
    

Loading annotations from: nabirds_with_loc_2019.json


In [19]:
it = iter(train_loader)
first = next(it)
print(len(train_loader))

357


In [20]:
first[0].shape[0]

64

In [21]:
from torchvision import models
import torch
import torch.nn as nn

In [24]:
model = models.inception_v3(pretrained=True)
model.fc = nn.Linear(2048, 555)
model.aux_logits = False
model = model.cuda()

In [25]:
lr =  0.0045
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr,
                            momentum=0.9,
                            weight_decay=1e-4)

In [29]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


In [30]:
def train(train_loader, model, criterion, optimizer, epoch):

    model.train()
    epoch_loss=0
    train_correct_total=0
    count=0
    for i, (input, im_id, target) in enumerate(train_loader):
        input_var = input.cuda()
        target_var = target.cuda()
        count+=input_var.shape[0]
        output = model(input_var)
        loss = criterion(output, target_var)
        epoch_loss += loss.item()
        predicted = torch.argmax(output, dim=1)
        correct_reviews_in_batch = (predicted == target_var).sum().item()
        train_correct_total += correct_reviews_in_batch
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch {} - Train Loss {:.2f}'.format(epoch + 1, epoch_loss ))
    print('Training Accuracy {:.4f}'.format(train_correct_total * 100 / count))


In [31]:
for epoch in tqdm(range(0, 3)):
    lr = lr * (0.1 ** (epoch // 3))
    train(train_loader, model, criterion, optimizer, epoch)

 33%|███▎      | 1/3 [07:46<15:32, 466.29s/it]

Epoch 1 - Train Loss 1475.46
Training Accuracy 17.9017


 67%|██████▋   | 2/3 [15:35<07:48, 468.14s/it]

Epoch 2 - Train Loss 1138.38
Training Accuracy 30.3782


100%|██████████| 3/3 [23:26<00:00, 468.76s/it]

Epoch 3 - Train Loss 951.26
Training Accuracy 39.2655


In [114]:
PATH ='/scratch/sm9669/nabird pickle/model_30_inception.pth'
torch.save(model.state_dict(), PATH)
